In [2]:
import sys
sys.path.append("../")
import nanonis
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import xlrd
import lmfit
from lmfit import Model
import matplotlib.colors as colors
%matplotlib qt
spectra=nanonis.biasSpectroscopy()

# Useful functions

In [3]:
def fermi(x,T):
    return np.divide(1,1+np.exp(x/T))


def BCS(x,delta,eta):
    return np.sign(x)*np.imag(np.divide(np.abs(x+1j*eta),np.sqrt(delta**2-(x+1j*eta)**2)))

def coulomb(x,Ec,Q0):
    return np.heaviside(x-Ec-Q0,0)+np.heaviside(-x-Ec+Q0,0)

def conv_dynes(x,delta1,delta2,eta,T,Ec,Q0):
    y=np.linspace(-5*(delta1+delta2),5*(delta1+delta2),5000)
    a,b=np.meshgrid(y,y)
    cou=np.gradient(-np.dot(BCS(a+b,delta1,eta)*fermi(a+b,T),coulomb(y,Ec,Q0))+np.dot(BCS(a+b,delta1,eta),fermi(y,T)*coulomb(y,Ec,Q0)))
    a,b=np.meshgrid(y,x)
    z=np.gradient(-np.dot(BCS(a+b,delta2,eta)*fermi(a+b,T),cou)+np.dot(BCS(a+b,delta2,eta),cou*fermi(y,T)))
    return z/z[0]

def temp(x,delta1,delta2,eta,T):
    y=np.linspace(-5*(delta1+delta2),5*(delta1+delta2),5000)
    a,b=np.meshgrid(y,x)
    return -np.dot(BCS(a+b,delta2,eta)*fermi(a+b,T),BCS(y,delta1,eta))+np.dot(BCS(a+b,delta2,eta),BCS(y,delta1,eta)*fermi(y,T))

def temp2(V,delta1,delta2,eta,T):
    y=np.linspace(-5*(delta1+delta2),5*(delta1+delta2),5000)
    return np.sum(-BCS(y+V,delta2,eta)*fermi(y+V,T)*BCS(y,delta1,eta)+BCS(y+V,delta2,eta)*BCS(y,delta1,eta)*fermi(y,T))

def gap(x,delta1,Bc1,Ec,a):
    d1=(0.0013*np.real(np.sqrt(1-((x+0.0000*1j)/800)**2)))*np.heaviside(-x+800,0)
    d2=delta1*np.real(np.sqrt(1-((x+0.0000*1j)/Bc1)**2))*np.heaviside(-x+Bc1,0)
    d3=(-a*x)*np.heaviside(x-Bc1,0)
    return Ec+d1+d2+d3
   
def coulomb_T(x,Ec,T):
    y=np.linspace(-3*np.min(x),3*np.min(x),2000)
    a,b=np.meshgrid(y,x)
    return np.gradient(-np.dot(fermi(a+b,T),coulomb(y,Ec,Q0))+fermi(y,T)*coulomb(y,Ec,Q0))




# File management #

In [242]:
data=pd.read_excel("C:/Users/jonor/Documents/GitHub/nanonis/Order.xls")
path='C:/Users/jonor/Desktop/PhD/Lanak/Pb on Gr (analysis)/2023.07.26 - 1st evap/'


In [244]:
Bfield_files=data.iloc[:,2]
Bfields=data.iloc[:,3]
size_dat=data.iloc[:,7]
height_dat=data.iloc[:,8]
temp=data.iloc[:,6]
temp_files=data.iloc[:,5]
josephson=data.iloc[:,4]

# B Field 

In [60]:
## All files
dat=[1,2,3,4,5,6,7,8,10,11,12,13,14,15,16,17,18,19,21,23,24]
BFields=[]
BFiles=[]
size=[]
height=[]
for i in dat:
    a=[int(j) for j in Bfields[i].split(',')]
    b=Bfield_files[i].split(',')    
    files=[path+i for i in b]
    BFields.append(a)
    BFiles.append(files)
    size.append(size_dat[i])
    height.append(height_dat[i])
size=np.array(size)
height=np.array(height)

In [734]:
i=12
a=[int(j) for j in Bfields[i].split(',')]
b=Bfield_files[i].split(',')
files=[path+i for i in b]


In [154]:
w_B=[0,1,2,3,4,5,6,7,8,10,11,12,13,14,15,16,17,18,19,21,23,24]
Bfiles=[]
size=[]
B=[]

for j in w_B:
    a=[int(i) for i in Bfields[j].split(',')]
    b=Bfield_files[j].split(',')
    files=[path+i for i in b]
    B.append(a)
    #field0=a.index(0)
    #files_B0.append(files[field0])
    Bfiles.append(files)
    size.append(size_dat[j])


### Plots

In [722]:
y=[]
for i in files:
    spectra.load(i)
    y.append(spectra.conductance/spectra.conductance[0])
x=np.array(spectra.bias)
plt.imshow(y,aspect='auto',origin='lower',extent=(1000*x[0],1000*x[-1],a[-1],a[0]))
plt.ylabel('B(mT)')
plt.xlabel('Bias (meV)')

Text(0.5, 0, 'Bias (meV)')

### Some info

We look at the B field point after 800 mT to see the effect of B field on the coulomb gap

In [216]:
def coulomb_T(x,Ec,Q0,T):
    return fermi(Ec-x+Q0,T)+fermi(Ec+x-Q0,T)
def coulomb(x,Ec,Q0):
    T=0.0003
    return fermi(Ec-x+Q0,T)+fermi(Ec+x-Q0,T)
def coulomb_T2(x,Ec,Q0,T):
    return np.sinh(np.abs(x)/T)/(np.cosh(x/T)+np.cosh(Ec/T))

h=0
Bfield=[800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300,2400,2500]
Gap=[]
Asym=[]
Si=[]
for k in Bfield:
    Ec=[]
    Q0=[]
    size2=[]
    for i in range(len(Bfiles)):
        for j in range(len(Bfiles[i])):
            if np.abs(B[i][j]-k)<=0.00001:
                size2.append(size[i])
                spectra.load(Bfiles[i][j])
                x=np.array(spectra.bias)
                y=np.array(spectra.conductance)
                y=y/y[0]
                #values=sc.optimize.curve_fit(coulomb_T,x,y,p0=[0.001,0.00,0.0003])[0]
                values=sc.optimize.curve_fit(coulomb,x,y,p0=[0.001,0.00])[0]
                #yy=coulomb_T(x,values[0],values[1],values[2])
                #yy=coulomb(x,values[0],values[1])
                Ec.append(values[0])
                Q0.append(values[1])
                #ther.append(values[2])
                #plt.plot(x,yy+h)
                #plt.plot(x,y+h)
                #h+=1
            else:
                continue
    Gap.append(Ec)
    Asym.append(Q0)
    Si.append(size2)

In [224]:
def coulomb_T(x,Ec,Q0,T):
    return fermi(Ec-x+Q0,T)+fermi(Ec+x-Q0,T)
def coulomb(x,Ec,Q0):
    T=0.0003
    return fermi(Ec-x+Q0,T)+fermi(Ec+x-Q0,T)
def coulomb_T2(x,Ec,Q0,T):
    return np.sinh(np.abs(x)/T)/(np.cosh(x/T)+np.cosh(Ec/T))

h=0
Bfield=[800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300,2400,2500]
Gap=[]
Asym=[]
Si=[]
for k in Bfield:
    Ec=[]
    Q0=[]
    size2=[]
    for i in range(len(Bfiles)):
        for j in range(len(Bfiles[i])):
            if np.abs(B[i][j]-k)<=0.00001:
                size2.append(size[i])
                spectra.load(Bfiles[i][j])
                x=np.array(spectra.bias)
                y=np.array(spectra.current)
                y=y/y[0]
                xx=[]
                for p in range(len(y)):
                    if np.abs(y[p]/y[0])<0.03:
                        xx.append(x[p])
                Ec.append(np.array(xx)[0]-np.array(xx)[-1])
            else:
                continue
        
    Gap.append(Ec)
    Asym.append(Q0)
    Si.append(size2)

In [227]:
plt.scatter(Si[12],Gap[12])

In [183]:
def coulomb(x,Ec,Q0):
    T=0.0003
    return fermi(Ec-x+Q0,T)+fermi(Ec+x-Q0,T)


emp=[]
for i in Bfiles:
    en=[]
    for j in i:
        en.append(0.0)
    emp.append(en)
Ec=emp
Q0=emp
for i in range(len(Bfiles)):
    for j in range(len(Bfiles[i])):
        spectra.load(Bfiles[i][j])
        x=np.array(spectra.bias)
        y=np.array(spectra.conductance)
        y=y/y[0]
        values=sc.optimize.curve_fit(coulomb,x,y,p0=[0.001,0.00])[0]
        #Ec[i][j]=values[0]
        Q0[i][j]=values[1]
        Ec[i][j]=values[0]
        
        


In [228]:
emp=[]
for i in Bfiles:
    en=[]
    for j in i:
        en.append(0.0)
    emp.append(en)
Ec=emp
Q0=emp
for i in range(len(Bfiles)):
    for j in range(len(Bfiles[i])):
        spectra.load(Bfiles[i][j])
        x=np.array(spectra.bias)
        y=np.array(spectra.current)
        y=y/y[0]
        xx=[]
        for p in range(len(y)):
            if np.abs(y[p]/y[0])<0.03:
                xx.append(x[p])
        Ec[i][j]=np.array(xx)[0]-np.array(xx)[-1]        

In [240]:
i=20
plt.plot(xx[i],yy[i])

In [241]:
xx=[]
yy=[]
for j in range(22):
    x=[]
    y=[]
    si=size[j]
    for i in range(len(B[j])):
        if B[j][i]>=1000:
            x.append(B[j][i])
            y.append(Ec[j][i])
        else:
            continue
    xx.append(x)
    yy.append(y)
    a,b=np.polyfit(x,y,1)
    #plt.plot(si,b,'o')

In [182]:

plt.plot(xx[j],yy[j])

In [241]:
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.viridis(np.linspace(0,1,10)))
#for i in range(18):
#    plt.scatter(Si[:][i],1000*np.array(Asym[:][i]))
plt.scatter(Si[0],1000*np.array(Asym[0]))
plt.scatter(Si[17],1000*np.array(Asym[17]))


#plt.scatter(size2,1000*np.array(Q08),c='#ff7f0e',label='T=8K')
#plt.xlabel('Island Area ($nm^2$)')
#plt.ylabel('$Q_0$ ($meV$)')
#plt.legend()

In [234]:
np.sum(np.abs(0-np.array(Asym[0])))/len(np.array(Asym[0])),np.sum(np.abs(0-np.array(Asym[17])))/len(np.array(Asym[17]))

(0.00021199756024640996, 0.00012050366485309387)

### Some info

The simplest way to get the gao is to calculate it from the current. We take the points for which the current is zero.



In [61]:
h=0
Ec=[]
Eg=[]
for k in range(len(BFiles)):
    cur=[]
    for i in BFiles[k]:
        spectra.load(i)
        cur.append(spectra.current)

    t=[]
    for j in range(len(BFiles[k])):
        y=np.array(cur[j])
        x=np.array(spectra.bias)
        xx=[]
        for i in range(len(y)):
            if np.abs(y[i]/y[0])<0.04:
                xx.append(x[i])
        t.append(xx[0]-xx[-1])
    T800=BFields[k].index(800)
    T0=BFields[k].index(0)
    Ec.append(t[T0])


plt.plot(size,Ec,'bo')


### Solme info

The critical field for the tip is around B=800mT, while the critical field for the island should be around B=350mT. Althought the first statement is prety robust, the second is not. 

The following program fit the spectra with a simple convolution. The free parameters will be the coulomb gap (Ec) the shift (Q0) and the dynes parameter (eta). We will suppose that the gaps decrease following the BCS theory with the critical fields that we wrote in the previous line. This first program is not a realistic fir of the system, we will only use it to get an idea of the volution of the full gap of the measurement, i.e., E+Delta1+Delta2. We will not make assupmtions about any of this parameters from the results of this analysis.

In [ ]:

def delta(Delta,B,Bc):
    return Delta*np.sqrt(1-(B/Bc)**2)

def conv_dynes(x,delta,eta,Ec):
    T=delta/10
    Q0=-0.00025
    y=np.linspace(-10*delta,10*delta,2000)
    a,b=np.meshgrid(y,x)
    z=np.gradient(-np.dot(BCS(a+b,delta,eta)*fermi(a+b,T),coulomb(y,Ec,Q0))+np.dot(BCS(a+b,delta,eta),fermi(y,T)*coulomb(y,Ec,Q0)))
    return z/z[0]


cond=[]
for i in files:
    spectra.load(i)
    cond.append(spectra.conductance)

gap=[]
for j in range(0,10):
    y=np.array(cond[j])[110:290]/np.array(cond[j])[110]
    x=np.array(spectra.bias)[110:290]
    temp=fit(conv_dynes,x,y,p0=[0.0013,0.0001,0.0012])
    plt.plot(x,y+0.5*j)
    p=temp[0]
    z=conv_dynes(x,p[0],p[1],p[2])
    plt.plot(x,z+0.5*j)
    gap.append(p[0]+p[2])
    print(j)

# Temperature dependence

In [101]:
data=[8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]
T=[]
files=[]
size=[]
height=[]

for j in data:
    a=temp[j].split(',')
    T.append([float(i) for i in a])
    b=temp_files[j].split(',')
    files.append([path+i for i in b])
    size.append(size_dat[j])
    height.append(height_dat[j])

In [102]:
f=files[0][0]
spectra.load(f)

x=np.array(spectra.bias)
y=np.array(spectra.conductance)
y=y/np.sum(y)


In [103]:
x=np.linspace(-0.01,0.01,1000)
plt.plot(x,coulomb_T(x,0.0005,0.0))

TypeError: coulomb_T() missing 1 required positional argument: 'T'

In [149]:
def coulomb_T(x,Ec,Q0,T):
    return fermi(Ec-x+Q0,T)+fermi(Ec+x-Q0,T)
def coulomb(x,Ec,Q0):
    T=0.0019455504685344104
    return fermi(Ec-x+Q0,T)+fermi(Ec+x-Q0,T)
def coulomb_T2(x,Ec,Q0,T):
    return np.sinh(np.abs(x)/T)/(np.cosh(x/T)+np.cosh(Ec/T))
Ec=[]
Q0=[]
ther=[]
h=0
temperature=8.0
size2=[]
for i in range(len(files)):
    for j in range(len(files[i])):
        if np.abs(T[i][j]-temperature)<=0.00001:
            size2.append(size[i])
            spectra.load(files[i][j])
            x=np.array(spectra.bias)
            y=np.array(spectra.conductance)
            y=y/y[0]
            #values=sc.optimize.curve_fit(coulomb_T,x,y,p0=[0.001,0.00,0.002])[0]
            values=sc.optimize.curve_fit(coulomb,x,y,p0=[0.001,0.00])[0]
            #yy=coulomb_T(x,values[0],values[1],values[2])
            yy=coulomb(x,values[0],values[1])
            Ec.append(values[0])
            Q0.append(values[1])
            #ther.append(values[2])
            #plt.plot(x,yy+h)
            #plt.plot(x,y+h)
            h+=1
        else:
            continue
        


plt.scatter(size2,1000*np.array(Ec),c='#ff7f0e',label='T=7K')

In [145]:
np.mean(ther)

0.0019455504685344104

In [177]:
Ec7
size7
size8
Ec8
Q08
Q07

In [153]:
plt.scatter(size7,1000*np.array(Q07),c='#1f77b4',label='T=7K')
x=np.linspace(100,2000,1000)
#plt.plot(x,1/(0.001*x))
plt.scatter(size8,1000*np.array(Q08),c='#ff7f0e',label='T=8K')
plt.xlabel('Island Area ($nm^2$)')
plt.ylabel('$E_C$ ($meV$)')
plt.legend()
#plt.scatter(np.array(size),1000*np.array(Q0),c='#ff7f0e')
#plt.scatter(np.array(size),1000*np.array(ther),c='#2ca02c')

In [175]:
np.sum(np.array(ther))/len(ther)

0.002530124760784503

# Josephson 

In [275]:
files=[]
a=[1,2,3,4,5,6,7,8,11,12,13,14,15,16,17,18,19,21,22,23,24]
for j in a:
    files.append([path+i for i in josephson[j].split(',')])
files
for i in files[6]:
    spectra.load(i)
    x=spectra.bias
    y=np.array(spectra.conductance)
    yt=np.sum(y)
    plt.plot(x,y/yt)

In [308]:
i=files[6][1]
spectra.load(i)
x=np.array(spectra.bias)[150:350]
y=np.array(spectra.conductance)[150:350]
yt=np.sum(y)
y=y/yt
peak=sc.signal.find_peaks(y)[0]
plt.plot(x,y)
plt.scatter(x[peak],y[peak])